# Data Load

In [1]:
import pandas as pd

In [2]:
from modules.data_store import data_store

There is data stored
Last data fetch completed


In [3]:
data_store = data_store()

Load the saved pickle file


In [4]:
trial = data_store.return_last_trial()
df = data_store.return_dict_df()[trial]
df.head()

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target
0,1,1.918119,CASE,08,00:00,27.680000,73.026667,466.750000,0.0,61.7,...,0.0,0.0,0.0,0.0,0.0,0.0,629.156667,381790.957381,0.001648,0.003161
1,1,1.918119,CASE,08,01:00,27.770000,73.120000,464.516667,0.0,61.7,...,0.0,0.0,0.0,0.0,0.0,0.0,627.106666,381790.957381,0.001643,0.003151
2,1,1.918119,CASE,08,02:00,27.766666,73.130001,466.233333,0.0,61.7,...,0.0,0.0,0.0,0.0,0.0,0.0,628.830000,381790.957381,0.001647,0.003159
3,1,1.918119,CASE,08,03:00,27.896666,73.488334,468.466667,0.0,61.7,...,0.0,0.0,0.0,0.0,0.0,0.0,631.551667,381790.957381,0.001654,0.003173
4,1,1.918119,CASE,08,04:00,27.921666,73.543333,471.550000,0.0,61.7,...,0.0,0.0,0.0,0.0,0.0,0.0,634.715000,381790.957381,0.001662,0.003189


# Target값 조정

In [6]:
sum_cols = [ '내부온도관측치',
 '내부습도관측치',
 'co2관측치',
 'ec관측치',
 '시간당분무량',
 '시간당백색광량',
 '시간당적색광량',
 '시간당청색광량',
 '시간당총광량']

In [7]:
df['sum'] = df[sum_cols].sum(axis=1)
df.head(1)

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum
0,1,1.918119,CASE,08,00:00,27.68,73.026667,466.75,0.0,61.7,61.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,629.156667


In [22]:
df[(df['DAT']==1)&(df['train/test']=='CASE')&(df['file_num']=='01')]['sum'].sum()

558111.7754508969

In [27]:
total_sum_df = df[['DAT','train/test','file_num','sum']].groupby(['DAT','train/test','file_num']).sum().reset_index()
total_sum_df.columns = ['DAT','train/test','file_num','total_sum']

base_df = pd.merge(df,total_sum_df,how='inner',on=['DAT','train/test','file_num'])
base_df['weight'] = base_df['sum'] / base_df['total_sum']
base_df['target'] = base_df['predicted_weight_g'] * base_df['weight']
base_df

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target
0,1,1.918119,CASE,08,00:00,27.680000,73.026667,466.750000,0.000000,61.7,...,0.0,0.0,0.0,0.0,0.0000,0.000,629.156667,381790.957381,0.001648,0.003161
1,1,1.918119,CASE,08,01:00,27.770000,73.120000,464.516667,0.000000,61.7,...,0.0,0.0,0.0,0.0,0.0000,0.000,627.106666,381790.957381,0.001643,0.003151
2,1,1.918119,CASE,08,02:00,27.766666,73.130001,466.233333,0.000000,61.7,...,0.0,0.0,0.0,0.0,0.0000,0.000,628.830000,381790.957381,0.001647,0.003159
3,1,1.918119,CASE,08,03:00,27.896666,73.488334,468.466667,0.000000,61.7,...,0.0,0.0,0.0,0.0,0.0000,0.000,631.551667,381790.957381,0.001654,0.003173
4,1,1.918119,CASE,08,04:00,27.921666,73.543333,471.550000,0.000000,61.7,...,0.0,0.0,0.0,0.0,0.0000,0.000,634.715000,381790.957381,0.001662,0.003189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22171,28,0.000000,TEST,02,19:00,26.960000,56.620000,542.616667,1.067239,769.0,...,0.0,0.0,0.0,0.0,12.3764,209656.216,1421.016706,446639.947445,0.003182,0.000000
22172,28,0.000000,TEST,02,20:00,26.211667,57.241667,540.583333,1.074064,0.0,...,0.0,0.0,0.0,0.0,0.0000,209656.216,625.110731,446639.947445,0.001400,0.000000
22173,28,0.000000,TEST,02,21:00,25.368333,56.310000,530.800000,1.078533,769.0,...,0.0,0.0,0.0,0.0,0.0000,209656.216,1382.556867,446639.947445,0.003095,0.000000
22174,28,0.000000,TEST,02,22:00,25.256667,57.343333,540.466667,1.078533,0.0,...,0.0,0.0,0.0,0.0,0.0000,209656.216,624.145200,446639.947445,0.001397,0.000000


In [32]:
def load_data(df,DAT,train_test,file_num):
    return df[(df['DAT']==DAT)&(df['train/test']==train_test)&(df['file_num']==file_num)]
load_data(base_df,28,'CASE','02')

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target
12072,28,65.505384,CASE,02,00:00:00.,25.600000,78.000000,660.000000,0.0,61.7,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,825.300000,246823.01124,0.003344,0.219030
12073,28,65.505384,CASE,02,01:00:00.,23.348333,78.896668,623.516667,0.0,61.7,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,787.461668,246823.01124,0.003190,0.208988
12074,28,65.505384,CASE,02,01:59:59.,23.138334,78.956667,624.100000,0.0,61.7,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,787.895001,246823.01124,0.003192,0.209103
12075,28,65.505384,CASE,02,02:59:59.,23.051667,78.820000,623.966667,0.0,61.7,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,787.538334,246823.01124,0.003191,0.209008
12076,28,65.505384,CASE,02,03:59:59.,22.896666,78.930001,624.633333,0.0,61.7,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,788.160001,246823.01124,0.003193,0.209173
12077,28,65.505384,CASE,02,04:59:59.,22.951666,79.093333,626.600000,0.0,61.7,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,790.345000,246823.01124,0.003202,0.209753
12078,28,65.505384,CASE,02,05:59:59.,23.748333,79.806667,605.366667,0.0,123.4,...,1087.2036,1087.2036,1029.1905,1029.1905,2116.3941,2116.3941,5065.109867,246823.01124,0.020521,1.344251
12079,28,65.505384,CASE,02,06:59:59.,25.805000,76.858335,545.550000,0.0,123.4,...,3536.3076,4623.5112,3347.6105,4376.8010,6883.9181,9000.3122,14539.449535,246823.01124,0.058906,3.858685
12080,28,65.505384,CASE,02,07:59:59.,27.921667,76.693333,515.933333,0.0,123.4,...,4878.3504,9501.8616,4618.0420,8994.8430,9496.3924,18496.7046,19736.733133,246823.01124,0.079963,5.238014
12081,28,65.505384,CASE,02,08:59:59.,27.823333,76.783334,539.300000,0.0,123.4,...,4881.6600,14383.5216,4621.1750,13616.0180,9502.8350,27999.5396,19772.976667,246823.01124,0.080110,5.247632


- data save

In [59]:
data_store.add_df(all_df = base_df)
data_store.save_pkl()

# Start EBM

In [5]:
from modules import config
config.input_cols

['DAT',
 'obs_time',
 '내부온도관측치',
 '내부습도관측치',
 'co2관측치',
 'ec관측치',
 '시간당분무량',
 '일간누적분무량',
 '시간당백색광량',
 '일간누적백색광량',
 '시간당적색광량',
 '일간누적적색광량',
 '시간당청색광량',
 '일간누적청색광량',
 '시간당총광량',
 '일간누적총광량']

In [9]:
set(base_df['train/test'])

{'CASE', 'TEST'}

In [10]:
train_df = base_df[base_df['train/test']=='CASE']
test_df = base_df[base_df['train/test']=='TEST']

In [11]:
from interpret.glassbox import ExplainableBoostingRegressor

X_train = train_df[config.input_cols]
y_train = train_df['target']

ebm = ExplainableBoostingRegressor()
ebm.fit(X_train, y_train)

ExplainableBoostingRegressor(feature_names=['DAT', 'obs_time', '내부온도관측치',
                                            '내부습도관측치', 'co2관측치', 'ec관측치',
                                            '시간당분무량', '일간누적분무량', '시간당백색광량',
                                            '일간누적백색광량', '시간당적색광량', '일간누적적색광량',
                                            '시간당청색광량', '일간누적청색광량', '시간당총광량',
                                            '일간누적총광량', 'DAT x 시간당총광량',
                                            'DAT x 시간당백색광량', 'DAT x 일간누적총광량',
                                            'DAT x obs_time', 'DAT x 시간당분무량',
                                            'DAT x 내부온도관측치', 'DAT x 일간누적분무량',
                                            '내부습도관측치 x 시간당총광량', 'DAT x co2관측치',
                                            'DAT x 일간누적백색광량'],
                             feature_...'continuous', 'categorical',
                                            'continuous', 'continuous',
                               

In [40]:
from interpret import show
from interpret.data import Marginal

marginal = Marginal().explain_data(X_train, y_train, name = 'Train Data')
show(marginal)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/interpret/visual/udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/interpret/visual/udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/interpret/visual/udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


<!-- http://127.0.0.1:7001/140611247210960/ -->

In [41]:
ebm_global = ebm.explain_global()
show(ebm_global)

<!-- http://127.0.0.1:7001/140611313965328/ -->

In [62]:
cols = config.input_cols

def get_eachCols_EBM_score(ebm_global, cols):
    EBM_cols_dict = dict()
    for i in range(len(cols)):
        EBM_cols_dict[cols[i]] = {'values':ebm_global.data(i)['names'],'scores':list(ebm_global.data(i)['scores'])}
    return EBM_cols_dict
EBM_cols_dict = get_eachCols_EBM_score(ebm_global, cols)
# EBM_cols_dict['DAT']

In [47]:
import numpy as np
def get_EBM_MinMax_scorePoint(df, EBM_cols_dict):
    EBM_Score_Min_max_df = dict()
    for k,v in EBM_cols_dict.items():
        EBM_Score_Min_max_df[k] = {'Unique':len(set(df[k])),'Max_Point':v['values'][v['scores'].index(max(v['scores']))],'Min_Point':v['values'][v['scores'].index(min(v['scores']))] ,'Max_Score' : int(max(v['scores'])), 'Min_Score' : int(min(v['scores'])), 'Midean' : int(np.median(v['scores']))}
    EBM_Score_Min_max_df = pd.DataFrame(EBM_Score_Min_max_df).transpose()
    EBM_Score_Min_max_df['Range_Score'] = EBM_Score_Min_max_df['Max_Score'] - EBM_Score_Min_max_df['Min_Score']
    return EBM_Score_Min_max_df.sort_values('Range_Score',ascending=False)
get_EBM_MinMax_scorePoint(train_df,EBM_cols_dict)



,Unique,Max_Point,Min_Point,Max_Score,Min_Score,Midean,Range_Score
DAT,28,27.5,3.5,2,0,0,2
co2관측치,14246,60.4,552.408,2,0,0,2
시간당총광량,2511,9242.35,36515.6,0,-2,0,2
obs_time,70,06:00,01:59:59.,0,-1,0,1
ec관측치,16306,3.65794,0.24145,0,-1,0,1
시간당분무량,1869,2259.73,123.4,1,0,0,1
시간당백색광량,1144,4641.15,8559.83,0,-1,0,1
시간당적색광량,888,1948.53,8687.7,1,0,0,1
일간누적적색광량,2595,154.724,20425.2,0,-1,0,1
시간당청색광량,960,9164.02,4621.17,1,0,0,1


해석해보자면 DAT은 클수록, co2관측치는 낮을수록 target값에 영향을 높게 준다.  
  
하지만 시간당 총광량은 9200수준이 36000수준보다 target에 영향을 높게 주는것으로 보인다.  이상하다.. 이부분을 EDA해 나가봐야겠다.  

> EDA 순서
> - 시간당 총광량
> - obs_time 확인 후 시간:00 으로 모두 고치기


In [55]:
base_df['시간당총광량'].describe()

count     22176.000000
mean       9272.340787
std        9608.289535
min     -165735.710800
25%           0.000000
50%        9242.350000
75%       18255.190000
max       37892.400000
Name: 시간당총광량, dtype: float64

- 시간당 총광량에 마이너스 값 존재 확인

# Base test predict

In [12]:
X_test = test_df[config.input_cols]
y_test = test_df['target']


test_df['predicted_weight_g'] = ebm.predict(X_test)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
test_df.head()

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target
18816,1,0.059478,TEST,05,00:00:00.,25.675,51.99,508.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,586.165,332711.959331,0.001762,0.0
18817,1,0.096088,TEST,05,01:00:00.,25.675,51.99,508.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,586.165,332711.959331,0.001762,0.0
18818,1,0.109053,TEST,05,02:00:00.,25.675,51.99,508.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,586.165,332711.959331,0.001762,0.0
18819,1,0.154528,TEST,05,03:00:00.,25.675,51.99,508.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,586.165,332711.959331,0.001762,0.0
18820,1,0.207238,TEST,05,04:00:00.,25.675,51.99,508.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,586.165,332711.959331,0.001762,0.0


In [14]:
base_rslt_df = test_df[['DAT','file_num','predicted_weight_g']].groupby(['DAT','file_num']).sum().reset_index()
def predict_rslt_weight(x):
    if x < 0 :
        return 0
    else : 
        return x

base_rslt_df['predicted_weight_g'] = base_rslt_df['predicted_weight_g'].apply(predict_rslt_weight)
# base_rslt_df = base_rslt_df.set_index('DAT')
base_rslt_df

,DAT,file_num,predicted_weight_g
0,1,01,6.263632
1,1,02,0.154602
2,1,03,0.000000
3,1,04,0.803110
4,1,05,0.018673
...,...,...,...
135,28,01,130.433610
136,28,02,79.700797
137,28,03,21.158445
138,28,04,136.753058


In [15]:
df.columns

Index(['DAT', 'predicted_weight_g', 'train/test', 'file_num', 'obs_time',
       '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량', '일간누적분무량', '시간당백색광량',
       '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량', '일간누적청색광량', '시간당총광량',
       '일간누적총광량', 'sum', 'total_sum', 'weight', 'target'],
      dtype='object')

In [23]:
file_num_list = set(base_rslt_df['file_num'])
test_target_list = [] 
file_path = './open/sample_submission/'
for file_num in file_num_list:
    test_target_file = file_path+'TEST_'+file_num+'.csv'
    print(test_target_file)
    test_target_list.append(test_target_file)
    base_rslt_df[base_rslt_df['file_num']==file_num][['DAT','predicted_weight_g']].to_csv(test_target_file,index=False)
    

./open/sample_submission/TEST_03.csv
./open/sample_submission/TEST_05.csv
./open/sample_submission/TEST_04.csv
./open/sample_submission/TEST_02.csv
./open/sample_submission/TEST_01.csv


In [36]:
# import zipfile
# import os
# os.chdir(file_path)
# submission = zipfile.ZipFile("./submission.zip", 'w')
# for path in test_target_list:
#     print(path)
#     path = path.split('/')[-1]
#     submission.write(path)
# submission.close()
# os.chdir('~')